In [1]:
#loading pre-trained face detecting model
import cv2
import numpy as np
face_det = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")

#Activating my webcam
cap = cv2.VideoCapture(0)  

#defining a function to detect and extract cropped faces
def face_ext(image):
    img_gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    img = face_det.detectMultiScale(img_gray, 1.3, 5)
    
    if img is ():
        return None
    
    for (x,y,w,h) in img:
        crop_face = image[y:y+h, x:x+w]
    
    return crop_face
  
#Creating dataset for training model for face detection
i=0
Training_Data, Labels = [], []
while i<100:
    ret, photo = cap.read()
    if face_ext(photo) is not None:
        face = cv2.resize(face_ext(photo),(200, 200))
        face = cv2.cvtColor(face,cv2.COLOR_BGR2GRAY)
        
        Training_Data.append(np.asarray(face, dtype=np.uint8))
        Labels.append(i)
        i += 1
        
       
        
        #displaying the dataset
        cv2.putText(face, str(i), (50, 50), cv2.FONT_HERSHEY_PLAIN, 1, (0,255,0), 2)
        cv2.imshow('Gathering Training Data', face)
    else:
        print("No face found")
        pass
    if cv2.waitKey(1)==13:
        break
cap.release()
cv2.destroyAllWindows()
Labels = np.asarray(Labels, dtype=np.int32)
print("Dataset Generated")
    
        



<>:14: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:14: SyntaxWarning: "is" with a literal. Did you mean "=="?
<ipython-input-1-43354a78a0b9>:14: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if img is ():


Dataset Generated


In [2]:
#Training our face detection model
model  = cv2.face_LBPHFaceRecognizer.create() 
model.train(np.asarray(Training_Data), np.asarray(Labels))
print("Model trained sucessefully")


Model trained sucessefully


In [3]:
#Testing face recognition model 
import os
import subprocess
import time

def face_detector(img, size=0.5):
    
    # Convert image to grayscale
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_det.detectMultiScale(gray, 1.3, 5)
    if faces is ():
        return img, []
    
    
    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,255),2)
        roi = img[y:y+h, x:x+w]
        roi = cv2.resize(roi, (200, 200))
    return img, roi


# Open Webcam
cap = cv2.VideoCapture(0)

while True:

    ret, frame = cap.read()
    image, face = face_detector(frame)
    try:
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)

        results = model.predict(face)
        
        if results[1] < 500:
            confidence = int( 100 * (1 - (results[1])/400) )
            disp_text = str(confidence) + '% Confident it is User'
            
        cv2.putText(image, disp_text, (100, 120), cv2.FONT_HERSHEY_COMPLEX, 1, (255,100,100), 2)
        
        if confidence > 80:
            cv2.putText(image, "Hey Divyanshu", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,100,255), 2)
            cv2.imshow('Face Recognition', image )
            
            #cv2.putText(image, "Hey anuj", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,255), 2)
            #cv2.imshow('Face Recognition', image )
            
            print("launching aws instance for you.....")
            
            os.system("aws ec2 create-key-pair --key-name NewKey")
            
            def instance_launch():
                i_id = subprocess.getoutput("aws ec2 run-instances  --region ap-south-1  --image-id ami-0e5d82cae7458738b  --instance-type t2.micro --count 1  --key-name NewKey  --query Instances[*].[InstanceId] --output text")
                print("EC2 Instance with instance id : {} created successfully".format(i_id))

            def volume_creation():
                az = input("Enter the availability zone of running instance.. : ")
                v_id = subprocess.getoutput("aws ec2 create-volume  --region ap-south-1  --volume-type gp2 --size 5 --availability-zone " + az + " --query VolumeId --output text")
                print("Created 5GB EBS Volume with volume id : {} created".format(v_id))
                time.sleep(15)
                
                
            def attach():
                i_id = input("Enter instance id: ")
                v_id = input("Enter volume id: ")
                os.system("aws ec2 attach-volume --device /dev/xvdf --instance-id  " + i_id +" --volume-id " + v_id + " ")
                print("EBS volume Attached successfully to the newly created instance")

            
            break
           
        
        else:
            cv2.putText(image, "Not sure who you are", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,100,255), 2)
            cv2.imshow('Face Recognition', image )

    except:
        cv2.putText(image, "No Face Detected", (220, 120) , cv2.FONT_HERSHEY_COMPLEX, 1, (0,111,255), 2)
        cv2.putText(image, "Looking for a face", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,100,255), 2)
        cv2.imshow('Face Recognition', image )
        pass
        
    if cv2.waitKey(1) == 13:
        break
    
        
cap.release()
cv2.destroyAllWindows()


#Launching an ec2 instance and a 5GB instance volume if second face is recognized:


<>:11: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:11: SyntaxWarning: "is" with a literal. Did you mean "=="?
<ipython-input-3-448a4f0f0ff2>:11: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if faces is ():


launching aws instance for you.....


In [4]:
instance_launch()
#attaching the EBS Volume to the EC2 Instance we just launched
#os.system("aws ec2 attach-volume --volume-id vol- --instance-id  --device /dev/sda1")

EC2 Instance with instance id : i-003762526c36165c5 created successfully


In [5]:
volume_creation()

Enter the availability zone of running instance.. : ap-south-1a
Created 5GB EBS Volume with volume id : vol-0d225d898b9ab5306 created


In [6]:
attach()

Enter instance id: i-003762526c36165c5
Enter volume id: vol-0d225d898b9ab5306
EBS volume Attached successfully to the newly created instance


In [7]:
thank you 

SyntaxError: invalid syntax (<ipython-input-7-5ec101f630f8>, line 1)